In [118]:
import sys, logging

%load_ext autoreload
%autoreload 2

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

import pandas as pd
pd.set_option('display.max_colwidth', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
# Assumes that full sap sam dataset was parsed already and stored in full_pickle.pkl
df = pd.read_pickle("full_pickle.pkl")

# Only use diagrams with no errors
df_no_err = df.query("not has_error").copy()
df_no_err

,has_error,shapes,edges,error_type,error_details
fname,,,,,
0000150e890c45928d26584d491ada68.bpmn,False,32,28,NaN,NaN
00001f477e9a4fb1b9dcb27e029a426d.bpmn,False,16,15,NaN,NaN
00003a67300748639e4bc21398bb5ede.bpmn,False,8,8,NaN,NaN
00005cea16fd426881c7f1619e30b804.bpmn,False,20,16,NaN,NaN
0000c3ea6c58407dbfaeac9ef5e467e7.bpmn,False,17,19,NaN,NaN
...,...,...,...,...,...
ffffc29ea95142e185189e9d35a51ad2.bpmn,False,28,25,NaN,NaN
ffffcd5186bf4655b510ec28d21f0dc3.bpmn,False,16,13,NaN,NaN
ffffe3bf630542ef846c139be1af7469.bpmn,False,19,18,NaN,NaN


In [120]:
# Eliminate standard example duplicates

df_no_err["shapes-edges"] = df_no_err.shapes.map(str) + "-" + df_no_err.edges.map(str)
se_vc = df_no_err["shapes-edges"].value_counts()
se_vc.sort_values(ascending=False).head(20)
cutoff = len(df_no_err) / 50
outliers = se_vc[se_vc > cutoff].index.tolist()
if "0-0" not in outliers:
    outliers.append("0-0")
outliers

mask = ~df_no_err["shapes-edges"].isin(outliers)
df_no_duplicates = df_no_err.loc[mask].reset_index()[["fname", "shapes", "edges"]]
df_no_duplicates

,fname,shapes,edges
0,0000150e890c45928d26584d491ada68.bpmn,32,28
1,00003a67300748639e4bc21398bb5ede.bpmn,8,8
2,00005cea16fd426881c7f1619e30b804.bpmn,20,16
3,0000c8488ca54f7cafe9556e9538defd.bpmn,65,57
4,00011e4ecbf740d4bd4dd71a5c845c4d.bpmn,49,60
...,...,...,...
285731,ffff335ece344fb0b2eb590cf9bbc720.bpmn,26,21
285732,ffffc29ea95142e185189e9d35a51ad2.bpmn,28,25
285733,ffffcd5186bf4655b510ec28d21f0dc3.bpmn,16,13
285734,ffffe3bf630542ef846c139be1af7469.bpmn,19,18


In [121]:
# Remove diagrams that have too many elements

shape_upper_limit = 200
edge_upper_limit = 200

df_final = df_no_duplicates[(df_no_duplicates['shapes'] <= shape_upper_limit) & (df_no_duplicates['edges'] <= edge_upper_limit)]

In [124]:
# Assumes that there is already a file with columns organization_id, model_id, split named organization_split.csv

df_organization_split = pd.read_csv("organization_split.csv")
df_organization_split['model_id'] = df_organization_split.model_id + '.bpmn'
df_model_split = pd.merge(df_final, df_organization_split, left_on='fname', right_on='model_id').rename(columns={"fname":"filename"})[["filename", "split"]]

df_model_split.to_csv("filename_split.csv", index=False)
df_model_split["split"].value_counts()

train    273728
test       6301
val        5184
Name: split, dtype: int64